In [1]:
import warnings
warnings.filterwarnings("ignore")
from tkinter import *
import tkinter.ttk as ttk
from PIL import Image,ImageTk
import os
import glob
from tkinter import filedialog
import pandas as pd
import numpy as np
import keras
import cv2
import csv
import pickle
import tensorflow as tf
from tkinter.filedialog import asksaveasfile
from keras.preprocessing.image import img_to_array,load_img
from tensorflow.python.keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 

print(tf.__version__)

Using TensorFlow backend.


2.3.1


In [2]:
cnn = tf.keras.models.load_model(r'Covid_result_Pretrained.h5')
dnn = tf.keras.models.load_model(r"Covid_result_DNN.h5")

In [3]:
a_file = open("FeatureReduction.pkl", "rb")
FeatureTransform = pickle.load(a_file)

In [4]:
path=''
files=[]

In [5]:
def csv():
    global path,files,df
    testCNN = []
    testDNN = []
    for f1 in files:
        res=test_image(path+"/"+f1)
        testCNN.append(res[0])
        testDNN.append(res[1])
    dic = {'IMAGE NAME':files, '--DNN_CLASSIFIER--':testDNN,'--CNN_CLASSIFIER--':testCNN,}
    df = pd.DataFrame(dic) 
    display(df)


In [6]:
def res():
    global path,cnn,dnn,l5,l6
    imlabel = (path.split('/')[3]).split('_')[1]
    res = test_image(path) 
    l5.config(text = "Predicted Label : \n\t"+res[1]+"\n\nActual Label : \n\t"+imlabel)
    l5.grid(row = 2,column = 1, pady = 2)
    l6.config(text = "Predicted Label : \n\t"+res[0]+"\n\nActual Label : \n\t"+imlabel)
    l6.grid(row = 2,column = 2, pady = 2)
    

In [7]:
class_dictionary = {'COVID': 0, 'NonCOVID': 1}
inv_map = { v: k for k, v in class_dictionary.items() }
def test_image(path):
    global inv_map,image
    image = read_image(path)
    img=path.split("/")[-1]
    features=FeatureTransform[img].reshape(1,-1)
    cnn_pred = cnn.predict(image)
    dnn_pred = dnn.predict(features)
    cnn_pred_label = np.argmax(cnn_pred, axis=1)
    dnn_pred_label = round(dnn_pred[0][0])
    txt_cnn = inv_map[cnn_pred_label[0]]
    txt_dnn = inv_map[dnn_pred_label]
    #print(txt_cnn,txt_dnn)
    return [txt_cnn,txt_dnn]

In [8]:
def sel():
    global b1,l1,b2,l2,l3,l4,l5,l6,TableMargin,rb1,rb2
    b1.grid(row = 0, column = 0,pady = 2)
    if v.get() == 1:
        rb1.config(bg='magenta')
        rb2.config(bg = "light blue")
        b1.config(text="Browse your folder",command=getDirectory)
        forgetlabels(l1,b2,l2,l3,l4,l5,l6,TableMargin)
    else:
        rb1.config(bg = "light blue")
        rb2.config(bg='magenta')
        TableMargin.pack_forget()
        b1.config(text="Browse your image",command=getImage)
        forgetlabels(l1,b2,l2,l3,l4,l5,l6,TableMargin)
def forgetlabels(l1,b2,l2,l3,l4,l5,l6,TableMargin):
    l1.grid_remove()
    b2.grid_remove()
    l2.grid_remove()
    l3.grid_remove()
    l4.grid_remove()
    lpic.grid_remove()
    l6.grid_remove()
    l5.grid_remove()
    f3.pack_forget()
    TableMargin.pack_forget()
def close():
    root.destroy()
def read_image(file_path):
    image = load_img(file_path, target_size=(224,224)) 
    image = img_to_array(image) 
    image = np.expand_dims(image, axis=0)
    image /= 255. 
    return image
def getImage():
    global l1,l2,l3,l4,b2,lpic,path,f3
    f3.pack(side = TOP)
    path = filedialog.askopenfilename(title='Select an Image')
    l1.config(text="PATH : "+path)
    image = Image.open(path).resize((200,200),Image.ANTIALIAS)
    photo = ImageTk.PhotoImage(image)
    lpic.config(image=photo)
    lpic.image = photo
    l1.grid(row = 0, column = 1, pady = 2)
    b2.config(command=res)
    b2.grid(row = 1, column = 1, pady = 2,sticky='w')
    l2.grid(row = 1, column = 0, pady = 2)
    l3.grid(row = 1, column = 1, pady = 2)
    l4.grid(row = 1, column = 2, pady = 2)
    lpic.grid(row = 2,column = 0, pady = 2)
def getDirectory():
    global path,files,l1
    path = filedialog.askdirectory()
    l1.config(text="PATH : "+path)
    l1.grid(row = 0, column = 1, pady = 2)
    files = os.listdir(path)
    path = path.replace("\\","/")
    b2.config(command=csv)
    b2.grid(row = 1, column = 1, pady = 2,sticky='w')

In [9]:
def display(df): 
    global TableMargin
    TableMargin.pack(side=TOP)
    scrollbarx = Scrollbar(TableMargin, orient=HORIZONTAL)
    scrollbary = Scrollbar(TableMargin, orient=VERTICAL)
    tree = ttk.Treeview(TableMargin,style="mystyle.Treeview", columns=("Image Name", "--DNN_CLASSIFIER--", "--CNN_CLASSIFIER--"), height=400, selectmode="extended", yscrollcommand=scrollbary.set, xscrollcommand=scrollbarx.set)
    style = ttk.Style()
    style.configure("mystyle.Treeview", highlightthickness=0, bd=0, font=('Calibri', 11)) # Modify the font of the body
    style.configure("mystyle.Treeview.Heading", font=('Calibri', 13,'bold')) # Modify the font of the headings
    style.layout("mystyle.Treeview", [('mystyle.Treeview.treearea', {'sticky': 'nswe'})]) # Remove the borders
    scrollbary.config(command=tree.yview)
    scrollbary.pack(side=RIGHT, fill=Y)
    scrollbarx.config(command=tree.xview)
    scrollbarx.pack(side=BOTTOM, fill=X)
    tree.heading('Image Name', text="Image Name", anchor= 'c')
    tree.heading('--DNN_CLASSIFIER--', text="-DNN_CLASSIFIER-", anchor= 'c')
    tree.heading('--CNN_CLASSIFIER--', text="-CNN_CLASSIFIER-", anchor= 'c')
    tree.column('#0', stretch=NO, minwidth=0, width=0, anchor= 'c')
    tree.column('#1', stretch=NO, minwidth=0, width=200, anchor= 'c')
    tree.column('#2', stretch=NO, minwidth=0, width=200, anchor= 'c')
    tree.column('#3', stretch=NO, minwidth=0, width=200, anchor= 'c')
    tree.tag_configure("evenrow",background='white',foreground='black')
    tree.tag_configure("oddrow",background='black',foreground='white')
        
    i = 0
    for row_index,row in df.iterrows():
        filename = row[0]
        dnn_label = row[1]
        cnn_label = row[2]
        tree.insert("", 0, values=(filename, dnn_label, cnn_label))
        if(i%2 == 0):
            tree.insert("", 0, values=(filename, dnn_label, cnn_label),tags = ('evenrow',))
        if(i%2 == 1):
            tree.insert("", 0, values=(filename, dnn_label, cnn_label),tags = ('oddrow',))
        i+=1
    #print(tree.tag_has('evenrow'))
    tree.pack()

In [ ]:
root = Tk()
root.geometry("900x600")
root.title("COVID DIAGONISIS")
v = IntVar()

f1 = Frame(root, width=100, height =50)
f1.pack(side = TOP)

f2 = Frame(root, width=100, height =50)
f2.pack(side = TOP)

f3 = Frame(root)
f3.pack(side = TOP)

TableMargin = Frame(root, width=500)

f4 = Frame(root)
f4.pack(side = TOP)

Label(f1).grid(row=0)
logo_image = ImageTk.PhotoImage(Image.open(r'sastra_logo.jpeg').resize((220, 40), Image.ANTIALIAS))
logo = Label(f1, image = logo_image,bd=1,padx=2,pady=2,bg='red',borderwidth=3, relief="flat", highlightcolor="white",anchor='w')
logo.grid(row=1,column=0)
title = Label(f1,text="COVID TESTER",width=15,fg='white',bg='purple',font=('arial',24,'bold'),borderwidth=3, relief="ridge").grid(row = 1, column = 1, pady = 2)
Label(f1).grid(row=2,pady=0.5)
label = Label(f1,text="""Choose an input option:""",width=20,justify = LEFT,fg='black',bg='white',font=('arial',12,'bold'), relief="ridge",anchor='w')
label.grid(row = 3, column = 0, pady = 2)
rb1 = Radiobutton(f1,text="Group of images",variable=v,value=1,command=sel,background = "light blue",relief='groove')
rb1.grid(row = 3, column = 1)
rb2 = Radiobutton(f1,text="A single image",variable=v,value=2,command=sel,background = "light blue",relief='groove')
rb2.grid(row = 3, column = 2,sticky='w')

b1 = Button(f2,bg='white',width=22,relief=RIDGE,font=('arial',12,'italic')) 
l1 = Label(f2,fg='black',bg='white',font=('arial',12,'bold'))
b2 = Button(f2,text='Analyse',bg='yellow',width=22,relief=RIDGE,font=('arial',12,'italic'))  
l2 = Label(f3,text="IMAGE",width=20,justify = LEFT,fg='black',bg='white',font=('arial',12,'bold'), relief="ridge")
l3 = Label(f3,text="DNN",width=15,justify = LEFT,fg='black',bg='white',font=('arial',12,'bold'), relief="ridge")
l4 = Label(f3,text="CNN",width=15,justify = LEFT,fg='black',bg='white',font=('arial',12,'bold'), relief="ridge")
lpic = Label(f3,bg='red')
l5 = Label(f3,fg='black',bg='white',font=('arial',12,'bold'), relief="ridge",pady=25)
l6 = Label(f3,fg='black',bg='white',font=('arial',12,'bold'), relief="ridge",pady=25)

Button(f2,text='EXIT',bg='red',width=22,relief=RIDGE,font=('arial',12,'italic'),command=close).grid(row = 1, column = 2, pady = 2,sticky='w')
root.mainloop()